In [1]:
import sklearn
from sklearn import datasets
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
intakes = pd.read_csv('Austin_Animal_Center_Intakes.csv',parse_dates=['DateTime','MonthYear'])
intakes.columns = intakes.columns.str.replace(" ","_")

In [3]:
outcomes = pd.read_csv('Austin_Animal_Center_Outcomes.csv',parse_dates=['DateTime','MonthYear','Date of Birth'])
outcomes.columns = outcomes.columns.str.replace(' ','_')

In [6]:
col_intersect = sorted(list(set(intakes.columns).intersection(set(outcomes.columns))))
col_intersect.remove("DateTime")
col_intersect.remove("MonthYear")

In [164]:
df = pd.merge(intakes, outcomes, on=col_intersect,suffixes=('_in', '_out') )
df.sort_index(axis=1,inplace=True)

In [8]:
print("intakes:    ",len(intakes))
print("outcomes:   ",len(outcomes))
print("total rows: ", len(intakes)+len(outcomes))
print("unique #:   ", len(df))

intakes:     123645
outcomes:    123983
total rows:  247628
unique #:    159728


In [165]:
df.drop(index=df[df.Age_upon_Outcome.isna()==True].index, inplace=True)
#MONTHYEAR IS SAME AS DATETIME
df.drop(columns=['MonthYear_in', 'MonthYear_out'],inplace=True)

In [166]:
# LOOK FOR WHEN DATETIME_IN IS EARLIER THAN DATE_OF_BIRTH
wrong_dob = (df.Date_of_Birth > df.DateTime_in)
#likely reasoning is DateTime_in is when the mother was brought into the center and the DOB is when the babies were born
#given this assumption, change all DateTime_in to DOB 
df.loc[wrong_dob, ['DateTime_in']] = df.loc[wrong_dob, ['Date_of_Birth']].values

In [167]:
# LOOK FOR AND SWITCH WHEN DATETIME_IN IS LATER THAN DATETIME_OUT
idx = (df.DateTime_out<df.DateTime_in)
#IF IN IS LATER THAN OUT, SWTICH
df.loc[idx,['DateTime_out','DateTime_in']] = df.loc[idx,['DateTime_in','DateTime_out']].values
#AGES APPEAR TO BE SWITCHED FOR ALL OF THESE OBSERVATIONS AS WELL. SWITCH THESE TWO COLUMNS
df.loc[idx,['Age_upon_Outcome', 'Age_upon_Intake']] = df.loc[idx,['Age_upon_Intake', 'Age_upon_Outcome']].values

In [168]:
# CREATE A TRUE DURATION IN DAYS FOR IN/OUT 
df['Duration_Days']= (df.DateTime_out - df.DateTime_in).dt.days 

In [169]:
#GIVEN AGES AREN'T RELIABLE. 
#CALCULATE AGE UPON INTAKE USING DATE OF BIRTH AND DATETIMEIN
#CALCULATE AGE UPON OUTCOME USING DATE OF BIRTH AND DATETIMEOUT
df['Calc_Age_In'] = round((df['DateTime_in'] - df['Date_of_Birth'])/ np.timedelta64(1, 'Y'),1)
df['Calc_Age_Out'] = round((df['DateTime_out'] - df['Date_of_Birth'])/ np.timedelta64(1, 'Y'),1)

In [170]:
#Create Alternative to Altered Status + Sex in each intake and outcome 
df['Altered'] = df['Sex_upon_Intake']!=df['Sex_upon_Outcome']
df['Sex'] = df.Sex_upon_Intake.str.split(" ").str[-1]
#Can drop sex_upon_ columns 

In [171]:
#Create Is Mixed Column
df['Mixed']= df.Breed.str.contains("Mix|/")

In [172]:
breed = pd.DataFrame(df[['Breed','Mixed']])
breed['Without_Mix'] = breed["Breed"].str.replace("Mix","")

df['Primary_Breed'] = breed["Without_Mix"].str.split("/", n = 2, expand = True)[0].str.rstrip()
df['Secondary_Breed'] = breed["Without_Mix"].str.split("/", n = 2, expand = True)[1]
df['Tertiary_Breed'] = breed["Without_Mix"].str.split("/", n = 2, expand = True)[2]


In [173]:
{i for i in df['Primary_Breed']}.union({i for i in df['Secondary_Breed']})

{'Abyssinian',
 'Affenpinscher',
 'Afghan Hound',
 'African',
 'Airedale Terrier',
 'Akbash',
 'Akita',
 'Alaskan Husky',
 'Alaskan Klee Kai',
 'Alaskan Malamute',
 'American',
 'American Bulldog',
 'American Curl Shorthair',
 'American Eskimo',
 'American Foxhound',
 'American Pit Bull Terrier',
 'American Sable',
 'American Shorthair',
 'American Staffordshire Terrier',
 'American Wirehair',
 'Anatol Shepherd',
 'Angora',
 'Angora-English',
 'Angora-French',
 'Angora-Satin',
 'Armadillo',
 'Australian Cattle Dog',
 'Australian Kelpie',
 'Australian Shepherd',
 'Australian Terrier',
 'Balinese',
 'Bantam',
 'Barred Rock',
 'Basenji',
 'Basset Hound',
 'Bat',
 'Beagle',
 'Bearded Collie',
 'Beauceron',
 'Bedlington Terr',
 'Belgian Hare',
 'Belgian Malinois',
 'Belgian Sheepdog',
 'Belgian Tervuren',
 'Bengal',
 'Bernese Mountain Dog',
 'Beveren',
 'Bichon Frise',
 'Birman',
 'Black',
 'Black Mouth Cur',
 'Bloodhound',
 'Blue Lacy',
 'Bluebird',
 'Bluetick Hound',
 'Bobcat',
 'Boerboel

In [174]:
{i for i in df['Primary_Breed']}.difference({i for i in df['Secondary_Breed']})

{'Afghan Hound',
 'African',
 'Akbash',
 'Alaskan Klee Kai',
 'American',
 'American Sable',
 'American Wirehair',
 'Angora-English',
 'Angora-French',
 'Armadillo',
 'Balinese',
 'Bantam',
 'Barred Rock',
 'Bat',
 'Bedlington Terr',
 'Belgian Hare',
 'Belgian Sheepdog',
 'Belgian Tervuren',
 'Beveren',
 'Birman',
 'Bluebird',
 'Bobcat',
 'Bombay',
 'Bouv Flandres',
 'Boykin Span',
 'Briard',
 'Britannia Petit',
 'Budgerigar',
 'Bunting',
 'Burmese',
 'Californian',
 'Canary',
 'Catbird',
 'Chartreux',
 'Checkered Giant',
 'Chickadee',
 'Chicken',
 'Chinchilla',
 'Chinchilla-Amer',
 'Chinchilla-Stnd',
 'Cinnamon',
 'Clumber Spaniel',
 'Cockatiel',
 'Cockatoo',
 'Cold Water',
 'Colorpoint',
 'Conure',
 'Cornish Rex',
 'Coton De Tulear',
 'Cottontail',
 'Coyote',
 'Crow',
 'Cymric',
 'Dachshund Stan',
 'Deer',
 'Devon Rex',
 'Dogo Argentino',
 'Dove',
 'Duck',
 'Dutch',
 'Dutch Sheepdog',
 'Dwarf Hotot',
 'Eng Toy Spaniel',
 'English Shepherd',
 'English Spot',
 'Exotic Shorthair',
 'Fer

In [175]:
df['Intake_Condition'].unique()

array(['Normal', 'Sick', 'Injured', 'Nursing', 'Aged', 'Medical', 'Other',
       'Feral', 'Pregnant', 'Behavior'], dtype=object)

In [176]:
df['Intake_Type'].unique()

array(['Stray', 'Owner Surrender', 'Public Assist', 'Wildlife',
       'Euthanasia Request', 'Abandoned'], dtype=object)

In [177]:
df.columns

Index(['Age_upon_Intake', 'Age_upon_Outcome', 'Animal_ID', 'Animal_Type',
       'Breed', 'Color', 'DateTime_in', 'DateTime_out', 'Date_of_Birth',
       'Found_Location', 'Intake_Condition', 'Intake_Type', 'Name',
       'Outcome_Subtype', 'Outcome_Type', 'Sex_upon_Intake',
       'Sex_upon_Outcome', 'Duration_Days', 'Calc_Age_In', 'Calc_Age_Out',
       'Altered', 'Sex', 'Mixed', 'Primary_Breed', 'Secondary_Breed',
       'Tertiary_Breed'],
      dtype='object')

In [217]:
basic = df[['Animal_Type','Calc_Age_In','Calc_Age_Out','Color','Breed', 'Duration_Days','Altered', 'Sex','Mixed']]

In [218]:
dog_basic = basic[basic['Animal_Type']=='Dog']

In [219]:
dog_basic.drop(columns='Animal_Type',inplace=True)

,Calc_Age_In,Calc_Age_Out,Color,Breed,Duration_Days,Altered,Sex,Mixed
0,2.0,2.0,Tricolor,Beagle Mix,4,False,Male,True
1,8.0,8.0,White/Liver,English Springer Spaniel,0,False,Female,False
2,1.0,1.0,Sable/White,Basenji Mix,6,True,Male,True
4,4.0,4.0,Tan/Gray,Doberman Pinsch/Australian Cattle Dog,3,False,Male,True
5,2.0,2.0,Chocolate,Labrador Retriever Mix,3,False,Male,True
...,...,...,...,...,...,...,...,...
159723,0.2,0.3,Brown Brindle/White,Bull Terrier Mix,58,True,Male,True
159724,0.2,0.3,Brown Brindle/White,Bull Terrier Mix,27,False,Male,True
159725,0.3,0.3,Brown Brindle/White,Bull Terrier Mix,19,False,Male,True
159726,2.0,2.0,Black,Labrador Retriever,0,False,Female,False


In [180]:
dog_basic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101609 entries, 0 to 159727
Data columns (total 10 columns):
Animal_ID        101609 non-null object
Animal_Type      101609 non-null object
Calc_Age_In      101609 non-null float64
Calc_Age_Out     101609 non-null float64
Color            101609 non-null object
Breed            101609 non-null object
Duration_Days    101609 non-null int64
Altered          101609 non-null bool
Sex              101608 non-null object
Mixed            101609 non-null bool
dtypes: bool(2), float64(2), int64(1), object(5)
memory usage: 7.2+ MB


In [199]:
dog_basic.drop(index=dog_basic[dog_basic['Sex'].isna()==True].index, inplace=True)

In [205]:
y = dog_basic['Duration_Days']
X = dog_basic.drop("Duration_Days", axis=1)

In [208]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sn
import statsmodels.discrete.discrete_model as sm

In [209]:
X

,Animal_ID,Animal_Type,Calc_Age_In,Calc_Age_Out,Color,Breed,Altered,Sex,Mixed
0,A786884,Dog,2.0,2.0,Tricolor,Beagle Mix,False,Male,True
1,A706918,Dog,8.0,8.0,White/Liver,English Springer Spaniel,False,Female,False
2,A724273,Dog,1.0,1.0,Sable/White,Basenji Mix,True,Male,True
4,A682524,Dog,4.0,4.0,Tan/Gray,Doberman Pinsch/Australian Cattle Dog,False,Male,True
5,A743852,Dog,2.0,2.0,Chocolate,Labrador Retriever Mix,False,Male,True
...,...,...,...,...,...,...,...,...,...
159723,A825881,Dog,0.2,0.3,Brown Brindle/White,Bull Terrier Mix,True,Male,True
159724,A825881,Dog,0.2,0.3,Brown Brindle/White,Bull Terrier Mix,False,Male,True
159725,A825881,Dog,0.3,0.3,Brown Brindle/White,Bull Terrier Mix,False,Male,True
159726,A828968,Dog,2.0,2.0,Black,Labrador Retriever,False,Female,False


In [ ]:
X_cons = sn.add_constant

do a column for AM/PM

In [76]:
df[df['Breed'].str.contains("/|-")]

,Age_upon_Intake,Age_upon_Outcome,Animal_ID,Animal_Type,Breed,Calc_Age_In,Calc_Age_Out,Color,DateTime_in,DateTime_out,...,Duration_Days,Found_Location,Intake_Condition,Intake_Type,Name,Outcome_Subtype,Outcome_Type,Sex_upon_Intake,Sex_upon_Outcome,Mixed
4,4 years,4 years,A682524,Dog,Doberman Pinsch/Australian Cattle Dog,4.0,4.0,Tan/Gray,2014-06-29 10:38:00,2014-07-02 14:16:00,...,3,800 Grove Blvd in Austin (TX),Normal,Stray,Rio,NaN,Return to Owner,Neutered Male,Neutered Male,True
24,2 months,2 months,A697950,Dog,Australian Cattle Dog/Labrador Retriever,0.2,0.2,Tan/White,2015-03-04 11:22:00,2015-03-08 18:55:00,...,4,1501 S Fm 973 in Austin (TX),Normal,Stray,NaN,NaN,Adoption,Intact Female,Spayed Female,True
114,6 months,1 year,A772747,Dog,Pit Bull/Australian Cattle Dog,0.5,1.1,White,2018-05-29 16:53:00,2019-01-07 13:41:00,...,222,Thaxton And Sassman in Austin (TX),Normal,Stray,Lamar,NaN,Adoption,Neutered Male,Neutered Male,True
115,1 year,1 year,A772747,Dog,Pit Bull/Australian Cattle Dog,1.1,1.3,White,2019-01-07 13:41:00,2019-02-28 14:43:00,...,52,Thaxton And Sassman in Austin (TX),Normal,Stray,Lamar,NaN,Adoption,Neutered Male,Neutered Male,True
116,1 year,2 years,A772747,Dog,Pit Bull/Australian Cattle Dog,1.1,2.9,White,2019-01-07 13:41:00,2020-10-04 15:46:00,...,636,Thaxton And Sassman in Austin (TX),Normal,Stray,Lamar,NaN,Rto-Adopt,Neutered Male,Neutered Male,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159672,2 years,2 years,A828706,Dog,Australian Shepherd/Border Collie,2.0,2.0,Black/White,2021-01-24 13:31:00,2021-01-30 13:50:00,...,6,9619 Braeburn Glen in Austin (TX),Normal,Stray,*Belvidere,NaN,Adoption,Intact Male,Neutered Male,True
159673,2 years,2 years,A828709,Dog,Australian Cattle Dog/Great Pyrenees,2.0,2.0,Tan/White,2021-01-24 15:21:00,2021-01-30 14:46:00,...,5,3302 Caleb Drive in Austin (TX),Normal,Stray,*Ripley,NaN,Adoption,Intact Male,Neutered Male,True
159674,2 years,2 years,A824483,Dog,Labrador Retriever/Pit Bull,2.0,2.0,Brown Brindle/White,2020-10-19 15:39:00,2020-10-26 19:31:00,...,7,15405 Long Vista Drive in Austin (TX),Normal,Stray,Ichabod,NaN,Adoption,Intact Male,Neutered Male,True
159695,1 year,1 year,A828879,Dog,Labrador Retriever/Pit Bull,1.0,1.0,Black/Brown,2021-01-27 18:43:00,2021-01-28 15:23:00,...,0,8220 West Sh 71 in Austin (TX),Normal,Stray,Zoe,NaN,Return to Owner,Intact Female,Intact Female,True


In [ ]:
# new data frame with split value columns 
new_col = aus["Color"].str.split("/", n = 1, expand = True) 
# making separate first name column from new data frame 
aus["Color_A"]= new_col[0]  
# making separate last name column from new data frame 
aus["Color_B"]= new_col[1] 

In [141]:
df['Color']

0                    Tricolor
1                 White/Liver
2                 Sable/White
3                      Calico
4                    Tan/Gray
                 ...         
159723    Brown Brindle/White
159724    Brown Brindle/White
159725    Brown Brindle/White
159726                  Black
159727            Black/White
Name: Color, Length: 159727, dtype: object

In [34]:
df.sort_index(axis=1,inplace=True)

In [187]:
dog = df[df.Animal_Type=="Dog"]

In [188]:
dog.Outcome_Type.value_counts()

Adoption           47176
Return to Owner    33052
Transfer           17768
Euthanasia          2080
Rto-Adopt           1152
Died                 280
Missing               51
Disposal              39
Name: Outcome_Type, dtype: int64

In [ ]:
dog

In [191]:
sorted(dog.Calc_Out_Age.unique())

[-3.1,
 -2.2,
 -1.7,
 -1.4,
 -1.1,
 -1.0,
 -0.8,
 -0.6,
 -0.3,
 0.0,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 1.0,
 1.1,
 1.2,
 1.3,
 1.4,
 1.5,
 1.6,
 1.7,
 1.8,
 1.9,
 2.0,
 2.1,
 2.2,
 2.3,
 2.4,
 2.5,
 2.6,
 2.7,
 2.8,
 2.9,
 3.0,
 3.1,
 3.2,
 3.3,
 3.4,
 3.5,
 3.6,
 3.7,
 3.8,
 3.9,
 4.0,
 4.1,
 4.2,
 4.3,
 4.4,
 4.5,
 4.6,
 4.7,
 4.8,
 4.9,
 5.0,
 5.1,
 5.2,
 5.3,
 5.4,
 5.5,
 5.6,
 5.7,
 5.8,
 5.9,
 6.0,
 6.1,
 6.2,
 6.3,
 6.4,
 6.5,
 6.6,
 6.7,
 6.8,
 6.9,
 7.0,
 7.1,
 7.2,
 7.3,
 7.4,
 7.5,
 7.6,
 7.7,
 7.8,
 7.9,
 8.0,
 8.1,
 8.2,
 8.3,
 8.4,
 8.5,
 8.6,
 8.7,
 8.8,
 8.9,
 9.0,
 9.1,
 9.2,
 9.3,
 9.4,
 9.5,
 9.6,
 9.7,
 9.8,
 9.9,
 10.0,
 10.1,
 10.2,
 10.3,
 10.4,
 10.5,
 10.6,
 10.7,
 10.8,
 10.9,
 11.0,
 11.1,
 11.2,
 11.3,
 11.4,
 11.5,
 11.6,
 11.7,
 11.8,
 11.9,
 12.0,
 12.1,
 12.2,
 12.3,
 12.4,
 12.5,
 12.6,
 12.7,
 12.8,
 12.9,
 13.0,
 13.1,
 13.2,
 13.3,
 13.4,
 13.5,
 13.6,
 13.7,
 13.8,
 13.9,
 14.0,
 14.1,
 14.2,
 14.3,
 14.4,
 14.5,
 14.6,
 14.7,
 

In [141]:
dog.Age_upon_Intake.unique()

array(['2 years', '8 years', '11 months', '4 years', '6 years',
       '5 months', '2 months', '18 years', '1 year', '4 months',
       '1 month', '3 years', '5 years', '6 months', '7 years',
       '10 months', '12 years', '10 years', '1 week', '7 months',
       '9 years', '14 years', '9 months', '8 months', '11 years',
       '4 weeks', '3 months', '3 weeks', '0 years', '15 years', '3 days',
       '13 years', '5 weeks', '17 years', '2 days', '2 weeks', '19 years',
       '1 day', '16 years', '6 days', '5 days', '4 days', '1 weeks',
       '20 years', '-1 years', '-3 years', '23 years', '-2 years',
       '24 years'], dtype=object)

,Age_upon_Intake,Age_upon_Outcome,Animal_ID,Animal_Type,Breed,Color,DateTime_in,DateTime_out,Date_of_Birth,Found_Location,Intake_Condition,Intake_Type,Name,Outcome_Subtype,Outcome_Type,Sex_upon_Intake,Sex_upon_Outcome,Duration_Days
0,2 years,2 years,A786884,Dog,Beagle Mix,Tricolor,2019-01-03 16:19:00,2019-01-08 15:11:00,01/03/2017,2501 Magin Meadow Dr in Austin (TX),Normal,Stray,*Brock,Partner,Transfer,Neutered Male,Neutered Male,4
1,8 years,8 years,A706918,Dog,English Springer Spaniel,White/Liver,2015-07-05 12:59:00,2015-07-05 15:13:00,07/05/2007,9409 Bluegrass Dr in Austin (TX),Normal,Stray,Belle,NaN,Return to Owner,Spayed Female,Spayed Female,0
2,11 months,1 year,A724273,Dog,Basenji Mix,Sable/White,2016-04-14 18:43:00,2016-04-21 17:17:00,04/17/2015,2818 Palomino Trail in Austin (TX),Normal,Stray,Runster,NaN,Return to Owner,Intact Male,Neutered Male,6
4,4 years,4 years,A682524,Dog,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,2014-06-29 10:38:00,2014-07-02 14:16:00,06/29/2010,800 Grove Blvd in Austin (TX),Normal,Stray,Rio,NaN,Return to Owner,Neutered Male,Neutered Male,3
5,2 years,2 years,A743852,Dog,Labrador Retriever Mix,Chocolate,2017-02-18 12:46:00,2017-02-21 17:44:00,02/18/2015,Austin (TX),Normal,Owner Surrender,Odin,NaN,Return to Owner,Neutered Male,Neutered Male,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159723,2 months,3 months,A825881,Dog,Bull Terrier Mix,Brown Brindle/White,2020-11-14 13:39:00,2021-01-11 16:55:00,09/13/2020,Teri Road And Pleasant Valley Road in Austin (TX),Normal,Stray,Bosco,NaN,Adoption,Intact Male,Neutered Male,58
159724,2 months,3 months,A825881,Dog,Bull Terrier Mix,Brown Brindle/White,2020-11-25 15:23:00,2020-12-23 13:10:00,09/13/2020,Austin (TX),Normal,Owner Surrender,Bosco,NaN,Adoption,Neutered Male,Neutered Male,27
159725,3 months,3 months,A825881,Dog,Bull Terrier Mix,Brown Brindle/White,2020-12-23 13:10:00,2021-01-11 16:55:00,09/13/2020,Austin (TX),Normal,Owner Surrender,Bosco,NaN,Adoption,Neutered Male,Neutered Male,19
159726,2 years,2 years,A828968,Dog,Labrador Retriever,Black,2021-01-30 08:14:00,2021-01-30 12:11:00,01/30/2019,1111 Bastrop Highway in Austin (TX),Normal,Public Assist,Rosie,NaN,Return to Owner,Intact Female,Intact Female,0


In [179]:
def transform_age(df, age_column):
    df = df.loc[df.loc[:, age_column] != 'NULL']
    
    range_column = age_column + '_Period Range'
    
    df[age_column + '_Periods'], df[range_column] = df.loc[:, age_column].str.split(' ').str[0].fillna(0).astype(int), df.loc[:, age_column].str.split(' ').str[1].fillna(0)
    
    
    df[range_column] = np.where(df[range_column].str.contains('day'), 1, 
                                  np.where(df[range_column].str.contains('week'), 7, 
                                           np.where(df[range_column].str.contains('month'), 30, 
                                                    np.where(df[range_column].str.contains('year'), 365, 0)))).astype(int)

    df[age_column + '_(days)'] = df[range_column] * df[age_column + '_Periods']
    df[age_column + '_(years)'] = df[age_column + '_(days)'] / 365
    
    df[age_column + '_age_group'] = pd.cut(df[age_column + '_(years)'], 10)

    return df